In [1]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

from llama_index.core import Settings, VectorStoreIndex, StorageContext
from chromadb.config import Settings as StChroma


from llama_index.llms.openai import OpenAI

# from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding


In [2]:
# configure the llm and the embedding model
llm = OpenAI(
    model="gpt-4o-mini",
    temperature=0.5,
)

Settings.llm = llm
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")


In [3]:
chroma_client = chromadb.PersistentClient(
    path="../chroma_recipe", settings=StChroma(allow_reset=True)
)
chroma_collection = chroma_client.get_or_create_collection(name="recipes")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)


index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    storage_context=storage_context,
)

# index = VectorStoreIndex(nodes, storage_context=storage_context)


In [4]:
query_engine = index.as_query_engine()


## Modifying the templates


In [5]:
# print(json.dumps(dict(query_engine.get_prompts()), indent=2))
import json

prompt_dict = query_engine.get_prompts()

# current templates
print(prompt_dict)


{'response_synthesizer:text_qa_template': SelectorPromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings={}, function_mappings={}, default_template=PromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='Context information is below.\n---------------------\n{context_str}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {query_str}\nAnswer: '), conditionals=[(<function is_chat_model at 0x7f3f4c9caa20>, ChatPromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, message_templates=[ChatMessage(role=<MessageRole.

In [29]:
# Create the selector fr the text_qa_template

from llama_index.core import PromptTemplate, SelectorPromptTemplate, ChatPromptTemplate
from llama_index.core.llms import ChatMessage

QA_TEMPLATE = """
'Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer:
"""

## Generate a basic chat message for the chat system
chat_sys = ChatMessage(
    content="""
You are an expert Q&A system that is trusted around the world.
Always answer the query using the provided context information, and not prior knowledge.
Some rules to follow:
1. Never directly reference the given context in your answer.
2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines.
""",
    role="system",
)

chat_user = ChatMessage(
    content="""
Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 
""",
    role="user",
)

text_qa_template = SelectorPromptTemplate(
    default_template=PromptTemplate(template=QA_TEMPLATE),
    conditionals=ChatPromptTemplate(message_templates=[chat_sys, chat_user]),
)


ValidationError: 1 validation error for SelectorPromptTemplate
conditionals
  value is not a valid list (type=type_error.list)

In [14]:
# Ref Template

REF_TEMPLATE = """
The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 
"""

refine_template = SelectorPromptTemplate(
    default_template=PromptTemplate(template=REF_TEMPLATE)
)


In [15]:
query_engine.update_prompts(
    {
        "response_synthesizer:text_qa_template": text_qa_template,
        "response_synthesizer:refine_template": refine_template,
    }
)


In [16]:
prompt_dict = query_engine.get_prompts()

# current templates
print(prompt_dict)


{'response_synthesizer:text_qa_template': SelectorPromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings={}, function_mappings={}, default_template=PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template="\n'Context information is below.\n---------------------\n{context_str}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {query_str}\nAnswer:\n"), conditionals=None), 'response_synthesizer:refine_template': SelectorPromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['query_str', 'existing_answer', 'context_msg'], kwargs={}, output_parser=None, template_var_mappings={}, function_mappings={}, default_template=PromptTemplate(metadata={'pr

In [17]:
resp = query_engine.query("Quien fue José Smith?")


In [18]:
print(resp.response)


No se proporciona información sobre José Smith en el contexto dado. El contenido se centra en recetas de salsas, específicamente Salsa Macha De Chile Cascabel y Autentica Salsa Macha. Si necesitas información sobre José Smith, te recomendaría buscar en fuentes históricas o biográficas.
